In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.ensemble import VotingClassifier
from catboost import CatBoostClassifier
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk


In [2]:
tbl = pd.read_csv(r'D:\hse_hw\news.csv')

In [ ]:
# Загрузка ресурсов NLTK
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
stop_words = set(stopwords.words('russian'))

# Функции для предобработки текста
def remove_patterns(text):
    """Очищает текст от символов, цифр, пунктуации и нежелательных паттернов."""
    # Удаление всего до вхождения '/' 
    text = re.sub(r'^.*?/', '', text)
    # Удаление упоминаний "РИА Новости" и двух следующих слов
    text = re.sub(r'\bРИА Новости\b(?:\s+\w+){1,2}', '', text)
    # Удаление упоминаний "Reuters" и двух следующих слов
    text = re.sub(r'\bReuters\b(?:\s+\w+){1,2}', '', text)
    # Удаление оставшейся пунктуации, символов, цифр
    text = re.sub(r'[^а-яё\s]', '', text.lower())
    return text.strip()

def clean_text(text):
    """Очищает текст от символов, цифр и пунктуации."""
    text = remove_patterns(text)
    text = re.sub(r'[^а-яё\s]', '', text.lower())
    return text.strip()

def preprocess_text(text):
    """Удаляет стоп-слова и лемматизирует текст."""
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token not in stop_words]
    return ' '.join(tokens)

def preprocess_data(df):
    """Проводит полную предобработку текста."""
    df = df.dropna(subset=['text'])
    df = df[df['text'].str.len() > 5]
    df['processed_text'] = df['text'].map(clean_text).map(preprocess_text)
    return df

# Предобработка данных
tbl = preprocess_data(tbl)

In [ ]:
# Преобразование текста в векторы
vectorizer = TfidfVectorizer(max_features=5000)
scaler = MaxAbsScaler()
X = vectorizer.fit_transform(tbl['processed_text'])
X = scaler.fit_transform(X)
y = tbl['topic']

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Определение пайплайнов для моделей
pipelines = {
    'RandomForest': Pipeline([('classifier', RandomForestClassifier(random_state=42))]),
    'ExtraTrees': Pipeline([('classifier', ExtraTreesClassifier(random_state=42))]),
    'SVM': Pipeline([('classifier', SVC(probability=True, random_state=42))]),
    'LogisticRegression': Pipeline([('classifier', LogisticRegression(max_iter=1000, random_state=42))]),
    'CatBoost': Pipeline([('classifier', CatBoostClassifier(verbose=0, random_state=42))]),
}

# Определение параметров для RandomizedSearchCV
param_grids = {
    'RandomForest': {
        'classifier__n_estimators': [50, 100, 200],
        'classifier__max_depth': [10, 20, 30],
        'classifier__min_samples_split': [2, 5, 10],
    },
    'ExtraTrees': {
        'classifier__n_estimators': [50, 100, 200],
        'classifier__max_depth': [10, 20, 30],
        'classifier__min_samples_split': [2, 5, 10],
    },
    'SVM': {
        'classifier__C': [0.1, 1, 10, 100],
        'classifier__kernel': ['linear', 'rbf', 'poly'],
    },
    'LogisticRegression': {
        'classifier__C': [0.1, 1, 10, 100],
        'classifier__penalty': ['l2'],
        'classifier__solver': ['lbfgs', 'liblinear']
    },
    'CatBoost': {
    'classifier__iterations': [100],
    'classifier__depth': [6, 8],
    'classifier__learning_rate': [0.1, 0.2]
    }
}

# RandomizedSearchCV для каждой модели
search_results = {}
metrics = []

for model_name, pipeline in pipelines.items():
    search = RandomizedSearchCV(pipeline, param_grids[model_name], n_iter=5, cv=3, scoring='accuracy', n_jobs=-1, random_state=42)
    search.fit(X_train, y_train)
    search_results[model_name] = search.best_estimator_

    # Оценка метрик
    y_pred = search.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    # Вывод classification_report
    print(f"\nClassification Report for {model_name}:")
    print(classification_report(y_test, y_pred))

    # Сохранение результатов
    metrics.append({'Model': model_name, 'Accuracy': accuracy, 'F1-score': f1})

# Блендинг моделей
voting_clf = VotingClassifier(
    estimators=[(name, model) for name, model in search_results.items()],
    voting='soft'
)
voting_clf.fit(X_train, y_train)

# Оценка метрик VotingClassifier
y_pred_voting = voting_clf.predict(X_test)
accuracy_voting = accuracy_score(y_test, y_pred_voting)
f1_voting = f1_score(y_test, y_pred_voting, average='weighted')

# Вывод classification_report для VotingClassifier
print("\nClassification Report for VotingClassifier:")
print(classification_report(y_test, y_pred_voting))

metrics.append({'Model': 'VotingClassifier', 'Accuracy': accuracy_voting, 'F1-score': f1_voting})

# Таблица метрик
metrics_df = pd.DataFrame(metrics)

# Отображение таблицы
display(metrics_df)

Classification Report for RandomForest:
              precision    recall  f1-score   support

         0.0       0.59      0.84      0.70      1394
         1.0       0.79      0.68      0.73       667
         2.0       0.82      0.85      0.84       647
         3.0       0.73      0.79      0.76       948
         4.0       0.97      0.94      0.95       417
         5.0       0.90      0.81      0.85       218
         6.0       0.84      0.63      0.72       984
         7.0       0.91      0.58      0.71       273
         8.0       0.93      0.35      0.51       320

    accuracy                           0.75      5868
   macro avg       0.83      0.72      0.75      5868
weighted avg       0.78      0.75      0.74      5868


Classification Report for ExtraTrees:
              precision    recall  f1-score   support

         0.0       0.56      0.88      0.69      1394
         1.0       0.81      0.64      0.71       667
         2.0       0.85      0.87      0.86       647
         3.0       0.75      0.75      0.75       948
         4.0       0.97      0.93      0.95       417
         5.0       0.94      0.73      0.82       218
         6.0       0.81      0.57      0.67       984
         7.0       0.92      0.59      0.72       273
         8.0       0.94      0.37      0.53       320

    accuracy                           0.73      5868
   macro avg       0.84      0.70      0.74      5868
weighted avg       0.77      0.73      0.73      5868


Classification Report for SVM:
              precision    recall  f1-score   support

         0.0       0.80      0.86      0.83      1394
         1.0       0.78      0.80      0.79       667
         2.0       0.88      0.92      0.90       647
         3.0       0.80      0.87      0.83       948
         4.0       0.97      0.98      0.97       417
         5.0       0.93      0.94      0.93       218
         6.0       0.76      0.62      0.68       984
         7.0       0.91      0.86      0.89       273
         8.0       0.89      0.82      0.86       320

    accuracy                           0.83      5868
   macro avg       0.86      0.85      0.85      5868
weighted avg       0.83      0.83      0.83      5868


Classification Report for LogisticRegression:
              precision    recall  f1-score   support

         0.0       0.82      0.87      0.84      1394
         1.0       0.80      0.81      0.81       667
         2.0       0.90      0.92      0.91       647
         3.0       0.82      0.86      0.84       948
         4.0       0.97      0.98      0.98       417
         5.0       0.95      0.94      0.95       218
         6.0       0.78      0.70      0.74       984
         7.0       0.92      0.86      0.89       273
         8.0       0.90      0.81      0.85       320

    accuracy                           0.84      5868
   macro avg       0.87      0.86      0.87      5868
weighted avg       0.84      0.84      0.84      5868



Classification Report for CatBoost:
              precision    recall  f1-score   support

         0.0       0.71      0.85      0.77      1394
         1.0       0.80      0.84      0.82       667
         2.0       0.88      0.87      0.87       647
         3.0       0.79      0.84      0.81       948
         4.0       0.97      0.94      0.96       417
         5.0       0.93      0.90      0.92       218
         6.0       0.99      0.76      0.86       984
         7.0       0.91      0.78      0.84       273
         8.0       0.92      0.67      0.77       320

    accuracy                           0.83      5868
   macro avg       0.88      0.83      0.85      5868
weighted avg       0.84      0.83      0.83      5868


Classification Report for VotingClassifier:
              precision    recall  f1-score   support

         0.0       0.82      0.88      0.85      1394
         1.0       0.83      0.83      0.83       667
         2.0       0.90      0.93      0.91       647
         3.0       0.82      0.88      0.85       948
         4.0       0.98      0.98      0.98       417
         5.0       0.94      0.94      0.94       218
         6.0       0.85      0.72      0.78       984
         7.0       0.91      0.87      0.89       273
         8.0       0.92      0.80      0.86       320

    accuracy                           0.86      5868
   macro avg       0.88      0.87      0.88      5868
weighted avg       0.86      0.86      0.86      5868

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Model</th>
      <th>Accuracy</th>
      <th>F1-score</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>RandomForest</td>
      <td>0.746080</td>
      <td>0.743718</td>
    </tr>
    <tr>
      <th>1</th>
      <td>ExtraTrees</td>
      <td>0.733810</td>
      <td>0.732001</td>
    </tr>
    <tr>
      <th>2</th>
      <td>SVM</td>
      <td>0.829073</td>
      <td>0.826704</td>
    </tr>
    <tr>
      <th>3</th>
      <td>LogisticRegression</td>
      <td>0.844751</td>
      <td>0.843857</td>
    </tr>
    <tr>
      <th>4</th>
      <td>CatBoost</td>
      <td>0.829755</td>
      <td>0.831549</td>
    </tr>
    <tr>
      <th>5</th>
      <td>VotingClassifier</td>
      <td>0.857532</td>
      <td>0.856638</td>
    </tr>
  </tbody>
</table>
</div>

Логистическая регрессия оказалась наилучшей моделью(не учитывая объединенеие моделей) для задачи классификации новостей, что может быть связано с её сбалансированными характеристиками и хорошей производительностью в случае классификации текстовых данных с дисбалансом классов.

Честно ожидал других результов, не могу предположить почему логистичекская регрессия оказалась лучше других моделей
Возможно еще что у CatBoost было очень мало гиперпарметров, я это сделал намеренно, так как обучение было слишком догим


### Как можно улучшить модель:
- использовать предобученные модели для превращения текста в новости 
- использовать DL
- Расширить диапазон гиперпараметров

In [ ]:
import joblib

# Сохранение модели VotingClassifier
joblib.dump(voting_clf, r'D:\hse_hw\final_voting_model.pkl')

print("Модель успешно сохранена в файл 'final_voting_model.pkl'.")

In [ ]:
file_path = r"D:\hse_hw\test_news.csv"
df = pd.read_csv(file_path)
df.head()


In [ ]:
df.rename(columns={'content': 'text'}, inplace=True) 
df = preprocess_data(df)
X = vectorizer.transform(df['processed_text'])
X = scaler.transform(X)
df['prediction'] = voting_clf.predict(X)
df = df.reset_index()
output_path = r"D:\hse_hw\test_news_predictions.csv"
df[['prediction', 'index']].to_csv(output_path, index=False)

print(f"Результаты предсказаний сохранены в {output_path}.")